In [1]:
import fiftyone as fo
from PIL import Image
from detection import detect

In [2]:
name = "dataset"
dataset_dir = "dataset"

In [3]:
# The splits to load
splits = ["val"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset(name)
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
    )

classes = dataset.default_classes

 100% |█████████████████| 640/640 [716.7ms elapsed, 0s remaining, 894.6 samples/s]      


In [4]:
# Do detections with model and save bounding boxes
with fo.ProgressBar() as pb:
    for sample in pb(dataset.view()):
        image = Image.open(sample.filepath)
        w, h = image.size
        pred = detect(sample.filepath, '../weights/yolo.onnx', '../weights/resnet.onnx')

        detections = []
        for _, row in pred.iterrows():
            xmin, xmax = int(row['xmin']), int(row['xmax'])
            ymin, ymax = int(row['ymin']), int(row['ymax'])
            rel_box = [
                xmin / w, ymin / h, (xmax - xmin) / w, (ymax - ymin) / h
            ]
            detections.append(
                fo.Detection(label=classes[int(row['cls'])],
                             bounding_box=rel_box,
                             confidence=int(row['cls_conf'])))

        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

 100% |█████████████████| 640/640 [8.8m elapsed, 0s remaining, 1.5 samples/s]      


In [5]:
dataset.persistent = True

In [6]:
results = dataset.view().evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |█████████████████| 640/640 [2.0s elapsed, 0s remaining, 319.5 samples/s]         
Performing IoU sweep...
 100% |█████████████████| 640/640 [2.1s elapsed, 0s remaining, 297.3 samples/s]      


In [7]:
# Print a classification report for all classes
results.print_report()

print(results.mAP())

# Plot confusion matrix
matrix = results.plot_confusion_matrix(classes=classes)
matrix.show()

pr_curves = results.plot_pr_curves(classes=classes)
pr_curves.show()

              precision    recall  f1-score   support

     Healthy       0.82      0.74      0.78       662
    Stressed       0.71      0.78      0.74       488

   micro avg       0.77      0.76      0.76      1150
   macro avg       0.77      0.76      0.76      1150
weighted avg       0.77      0.76      0.77      1150

0.6225828327927432


FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': 'c432eebe-8bf5-4b15-834c-abdc3af7c18b',
              'x': array([0, 1, 2, 0, 1, 2, 0, 1, 2]),
              'y': array([0, 0, 0, 1, 1, 1, 2, 2, 2])},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '01498ed3-dbe3-4c9e-baf9-49d814522d20',
              'z': array([[105, 158,   0],
                          [  0, 382, 106],
                          [493,   0, 169]]),
              'zmax': 493,
        

FigureWidget({
    'data': [{'customdata': array([99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 98.8,
                                   98. , 98. , 97.7, 97. , 97. , 97. , 96.9, 96.4, 96. , 95.7, 94.8, 94.3,
                                   93.8, 92.8, 92.2, 91.5, 90.9, 90.7, 89.3, 88.3, 87.8, 87.1, 86.8, 86.3,
                                   85.8, 85.1, 84.6, 84.1, 83.2, 82. , 81.7, 80.8, 79.7, 79.4, 78.6, 77.8,
                                   77.4, 76.3, 75.5, 74.6, 73.6, 72.3, 71. , 69.9, 68.6, 67.5, 66.4, 65.4,
                                   64.3, 63.2, 62.2, 61. , 60.2, 59.1, 58.1, 56.9, 50.4, 49. , 47.3, 46.4,
                                   40.8, 25.3, 10. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
                                    0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
                                    0. ,  0. ,  0. ,  0. ,  0. ]),
              'hovertemplate': ('<b>class: %{text}</b><br>reca

In [8]:
session = fo.launch_app(dataset, auto=False)
session.view = dataset.view()
session.plots.attach(matrix)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [9]:
def export_dataset(dataset, export_dir):
    label_field = "ground_truth"

    # The splits to export
    splits = ["val"]

    classes = ["Healthy", "Stressed"]

    # Export the splits
    for split in splits:
        split_view = dataset.match_tags(split)
        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            split=split,
            classes=classes,
        )